In [1]:
import os
import ee
import geemap

from concurrent.futures import ThreadPoolExecutor

In [ ]:
ee.Authenticate()
ee.Initialize(project="ee-bytesizedagric")
print(ee.String('Hello from the Earth Engine servers!').getInfo())

In [ ]:
# Get Côte d’Ivoire geometry
aoi = ee.FeatureCollection("FAO/GAUL/2015/level0") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Côte d\'Ivoire')) \
    .geometry()

# Set time range
start_date = '2024-01-01'
end_date = '2024-12-31'

# Folder to save images
save_folder = 'sentinel-2'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Select agriculture-relevant bands
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

# Filter Sentinel-2 SR image collection
collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# Convert to list
image_list = collection.toList(collection.size())

# Get the number of images
image_count = collection.size().getInfo()

# Download function (silent)
def download_image(i):
    try:
        image = ee.Image(image_list.get(i))
        image_id = image.id().getInfo().replace('/', '_')
        filename = os.path.join(save_folder, f"{image_id}.tif")

        if not os.path.exists(filename):
            geemap.download_ee_image(
                image=image.select(bands),
                filename=filename,
                region=aoi,
                scale=10000,
                crs='EPSG:4326'
            )
    except:
        pass  # Silently skip any errors

# Run in parallel
MAX_THREADS = 5  # Tune as needed
MAX_DOWNLOADS = image_count

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(download_image, range(min(image_count, MAX_DOWNLOADS)))